### Import packages

In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from scipy.stats import stats
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

### Create function to calculate pvalues from a sklearn linear regression
- With thanks to JARH from stack overflow <br>
https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression

In [2]:
def get_pvals(X_train, y_train):
    cols = np.array(X_train.columns.values)
    cols = np.insert(cols, 0, 'coef', axis=0)

    lm = LinearRegression()
    lm.fit(X_train, y_train)
    params = np.append(lm.intercept_,lm.coef_)
    predictions = lm.predict(X_train)

    newX = np.append(np.ones((len(X_train),1)), X_train, axis=1)
    MSE = (sum((y_train - predictions)**2)) / (len(newX) - len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)

    pvals = pd.DataFrame()
    pvals['feature'], pvals['coef'], pvals['standard_error'], pvals['t_vals'], pvals['pval'] = \
        [cols, params,sd_b,ts_b,p_values]
    
    return pvals

### Function to identify highest pvalue from an OLS summary table, used later for feature elemination

In [17]:
def get_kill_col(X_train, y_train):
    # Get pvalues from get_pvals
    pval = get_pvals(X_train, y_train)
    
    # Identify column with highest pvalue
    kill_cols = pval[pval['pval'] == pval['pval'].max()]
    highest_p_col = kill_cols['feature'].values
    highest_p = kill_cols['pval'].values
    
    return highest_p_col[0], highest_p[0]

### Function to create an OLS linear regression

In [18]:
def onetime_ols(X_train, y_train):
    # Create x constants
    Xconst = sm.add_constant(X_train)

    # Create OLS model and summary
    ols_model = sm.OLS(y_train, Xconst, hasconst= True)
    model = ols_model.fit()
    results = model.summary()
    
    return model, results

### Retrieve training data

In [83]:
X_train = pd.read_csv('../data/X_train2.csv', index_col=0)

pickle_in = open('../data/y_train.pickle', 'rb')
y_train = pickle.load(pickle_in)

# y_train = np.log(y_train)

In [84]:
y_log = np.log(y_train)

In [85]:
y_test = np.exp(y_log)

In [86]:
y_test[1]

798999.9999999998

In [87]:
y_train[1]

799000.0

### Loop get_kill_col to recursively remove lest significant features to model
- Loop terminates once the largest p-value in the regression is less than 0.05

In [88]:
xt = X_train
yt = y_train
p = 1
run = 1
results = {}
kill_cols = []

while p > 0.05:
        run_num = 'run{}'.format(run)
        high_col, high_p = get_kill_col(xt, yt)
        xt = xt.drop(columns=high_col)
        print('{}, column is {} with pval of {}'.format(run_num, high_col, high_p))
        kill_cols.append(high_col)
        run += 1
        p = high_p

results;

run1, column is zipcode_98022 with pval of 0.05


### Capture columns to remove from the model and remove them

In [89]:
kill_cols

X_train2 = X_train.drop(columns=kill_cols)